In [60]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import copy
import pandas as pd
import random
import pickle
import importlib
import datetime

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

import EEGNet as eegnet
importlib.reload(eegnet)

<module 'EEGNet' from '/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/model_training/EEGNet.py'>

In [61]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [62]:
# import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


## Dataset

In [63]:
class EarthquakeData(Dataset):
    def __init__(self, h_path, d_path):
        self.c_path = h_path + d_path
        self.h_len = len(h_path)
    
    def __len__(self):
        return len(self.c_path)
    
    def __getitem__(self, idx):
        path = self.c_path[idx]

        if idx > self.h_len:
            y = 1
        else:
            y = 0
            
        X = np.loadtxt(path, delimiter=',', dtype=str).astype(np.float32)

        return X, y

In [64]:
data_directory = 'area2-detrended'
# second_data_directory = 'area2-all'
# third_data_directory = 'area3-all'

class1_paths = glob.glob("/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/58daysdata/" + data_directory + "/nonSSE/*.csv")
class2_paths = glob.glob("/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/58daysdata/"  + data_directory + "/SSE/*.csv")

# class1_paths_second = glob.glob("/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/58daysdata/" + third_data_directory + "/nonSSE/*.csv")
# class2_paths_second = glob.glob("/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/58daysdata/"  + third_data_directory + "/SSE/*.csv")

# class1_paths = class1_paths + class1_paths_second
# class2_paths = class2_paths + class2_paths_second

# class1_paths_third = glob.glob("/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/58daysdata/" + third_data_directory + "/nonSSE/*.csv")
# class2_paths_third = glob.glob("/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/58daysdata/"  + third_data_directory + "/SSE/*.csv")

# class1_paths = class1_paths + class1_paths_third
# class2_paths = class2_paths + class2_paths_third

print("Before Undersampling")
print("NonSSE:", len(class1_paths))
print("SSE:", len(class2_paths))

undersample = True
if undersample:
    if class1_paths > class2_paths:
        sampled = random.sample(range(len(class1_paths)), len(class2_paths))
        new_class1_paths = []
        for x in sampled:
            new_class1_paths.append(class1_paths[x])
        class1_paths = new_class1_paths

print(" ")
print("After Undersampling")
print("NonSSE:", len(class1_paths))
print("SSE:", len(class2_paths))


Before Undersampling
NonSSE: 3306
SSE: 2136
 
After Undersampling
NonSSE: 2136
SSE: 2136


This code partitions the code in four sections, for testing on unseen time scale

In [65]:
crossvalidationsets = []

key_dates = [[datetime.datetime(1996, 12, 12), datetime.datetime(2000,9,24)], 
             [datetime.datetime(2000,9,25), datetime.datetime(2004,7,7)], 
             [datetime.datetime(2004,7,8), datetime.datetime(2008,4,19)], 
             [datetime.datetime(2008,4,20), datetime.datetime(2012,1,2)]]

# Divide the data into two different sections
for x in range(0, len(key_dates)):
    sse_train = []
    nonsse_train = []
    sse_validation = []
    nonsse_validation = []

    for address in class1_paths:
        first_split = address.split(":")[0].split("/")
        start_date = first_split[len(first_split)-1]
        s_date = start_date.split("-")
        current_start = datetime.datetime(int(s_date[0]), int(s_date[1]), int(s_date[2]))
        if key_dates[x][0] <= current_start <= key_dates[x][1]:
            sse_validation.append(address)
        else:
            sse_train.append(address)

    for address in class2_paths:
        first_split = address.split(":")[0].split("/")
        start_date = first_split[len(first_split)-1]
        s_date = start_date.split("-")
        current_start = datetime.datetime(int(s_date[0]), int(s_date[1]), int(s_date[2]))
        if key_dates[x][0] <= current_start <= key_dates[x][1]:
            nonsse_validation.append(address)
        else:
            nonsse_train.append(address)

    # Create the training dataset
    training_dataset = EarthquakeData(sse_train, nonsse_train)

    # Put validation and test data into dataset and split 50/50
    validationtest_dataset = EarthquakeData(sse_validation, nonsse_validation)
    val_len = len(validationtest_dataset)
    train_size = int(val_len * 0.5)
    val_size = val_len - train_size
    validation_dataset, test_dataset = torch.utils.data.random_split(validationtest_dataset, [val_size, train_size])

    # Put everything into array
    crossvalidationsets.append([training_dataset, test_dataset, validation_dataset])

    print("Iteration: ", x)
    print("Training")
    print("SSE: " + str(len(sse_train)) + " nonSSE: " + str(len(nonsse_train)))
    print("Validation and Testing")
    print("SSE: " + str(len(sse_validation)) + " nonSSE: " + str(len(nonsse_validation)))
    print()
    

Iteration:  0
Training
SSE: 1512 nonSSE: 1746
Validation and Testing
SSE: 624 nonSSE: 390

Iteration:  1
Training
SSE: 1664 nonSSE: 1515
Validation and Testing
SSE: 472 nonSSE: 621

Iteration:  2
Training
SSE: 1594 nonSSE: 1552
Validation and Testing
SSE: 542 nonSSE: 584

Iteration:  3
Training
SSE: 1638 nonSSE: 1595
Validation and Testing
SSE: 498 nonSSE: 541



This is where we train the model 4 times of different datasets

In [67]:
statistics = []

for datasets in crossvalidationsets:
    
    # Creates training, validation, and test data for each training dataset
    batch_size = 8
    dataloader = DataLoader(datasets[0], batch_size, shuffle=True)
    test_dataloader = DataLoader(datasets[1], batch_size, shuffle=True)
    validation_dataloader = DataLoader(datasets[2], batch_size, shuffle=True)

    # Sets up the parameters of the model
    num_chans = 30
    model = eegnet.EEGNet(Chans = num_chans, Samples = 128, nb_classes=1, kernLength=5).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Set Epochs and train the model
    epochs = 50
    best_weights, accuracy_stats, loss_stats = eegnet.train(model, optimizer, criterion, dataloader, 
                                                    validation_dataloader, epochs=epochs, device=device)
    
    # Load in best weights for the model for testing
    model.load_state_dict(best_weights)

    # Grab the testing score of the model
    test_acc = 0
    for test_features, test_labels in test_dataloader:

        test_features, test_labels = test_features.to(device), test_labels.to(device)
        test_features = test_features.float()
        test_labels = test_labels.float()
        
        test_pred = model(test_features)
        test_pred = torch.squeeze(test_pred)
        test_acc_item = eegnet.binary_acc(test_pred, test_labels)
        test_acc += test_acc_item.item()
    
    final_test_acc = test_acc/len(test_dataloader)
    print("Testing Accuracy: ", final_test_acc)

    statistics.append([best_weights, accuracy_stats, loss_stats, final_test_acc, model])



Iteration: 0 / 408
Iteration: 100 / 408
Iteration: 200 / 408
Iteration: 300 / 408
Iteration: 400 / 408
Epoch 000: | Train Loss: 0.71606 | Val Loss: 0.71701 | Train Acc: 52.821 | Val Acc: 50.844
Iteration: 0 / 408
Iteration: 100 / 408
Iteration: 200 / 408
Iteration: 300 / 408
Iteration: 400 / 408
Epoch 001: | Train Loss: 0.70814 | Val Loss: 0.72552 | Train Acc: 54.216 | Val Acc: 48.984
Iteration: 0 / 408
Iteration: 100 / 408
Iteration: 200 / 408
Iteration: 300 / 408
Iteration: 400 / 408
Epoch 002: | Train Loss: 0.69539 | Val Loss: 0.74101 | Train Acc: 55.877 | Val Acc: 46.344
Iteration: 0 / 408
Iteration: 100 / 408
Iteration: 200 / 408
Iteration: 300 / 408
Iteration: 400 / 408
Epoch 003: | Train Loss: 0.69517 | Val Loss: 0.74752 | Train Acc: 54.120 | Val Acc: 46.078
Iteration: 0 / 408
Iteration: 100 / 408
Iteration: 200 / 408
Iteration: 300 / 408
Iteration: 400 / 408
Epoch 004: | Train Loss: 0.67787 | Val Loss: 0.76390 | Train Acc: 58.966 | Val Acc: 42.000
Iteration: 0 / 408
Iteration: 

Record the statistics of each model

In [68]:
model_tested = "area2-detrended/"
file_names = ['first', "second", "third", "fourth"]

directory_path = "/Users/haxby/Desktop/Earthquakes/gnss-sse-detection-main/cross_validation_results/"
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

directory_path = directory_path + model_tested
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# add the statistics to each file
for i, dir_name in enumerate(file_names):
    final_path = directory_path + dir_name + "/"

    if not os.path.exists(final_path):
        os.makedirs(final_path)
    
    # Accuracy Scores
    scores_file = open(final_path + "accuracy_scores.txt", 'w')
    scores_file.write("Testing Score: " + str(statistics[i][3]) + "\n")
    scores_file.close()

    # Save the model
    torch.save(statistics[i][4].state_dict(), final_path+"saved_model.pth")

    # Save Accuracy Statistics
    acc_file = open(final_path + "accuracy_stats.obj", 'wb')
    pickle.dump(statistics[i][1], acc_file)
    acc_file.close()

    loss_file = open(final_path+"loss_stats.obj", 'wb')
    pickle.dump(statistics[i][2], loss_file)
    loss_file.close()

    b_weights_file = open(final_path+"best_weights.obj", "wb")
    pickle.dump(statistics[i][0], b_weights_file)
    b_weights_file.close()
